# Project 2: Continuous control

In [2]:
from IPython.display import HTML
style = "<style>.pseudo-code ul { list-style-type: none;}</style>"
HTML(style)

## 1: Learning algorithm

### Description of algorithm

Generally, in Q-learning, the goal is to find the true underlying Q-values, represented by $q^*$. In Deep Q-learning, neural networks are used to approximate $q^*$. The objective is then to find the weights, $w$, of the Q-network which best approximates $q^*$. That is, to minimize

\begin{equation} \min_w (q^*(s,a) - q(s,a;w))^2  \end{equation}

The Double Q-Network model with an Experience Buffer and Fixed Q-targets can be summarized as follows:

1. We start by creating a Replay Buffer (for Experience Replay) of size BUFFER_SIZE. This means that we will save the last BUFFER_SIZE experiences consisting of ($s_t$, $a_t$, $r_t$, $s_{t+1}$, $d_t$) tuples where 
    * $s_t$ is the state at step t,
    * $a_t$ is the action chosen at step t,
    * $r_t$ is the reward received after taking action $a_t$ at state $s_t$,
    * $s_{t+1}$ is the resulting next state, and
    * $d_t$ which is True if we ended up in a terminal state.
    
The motivation for the Replay Buffer is to break the correleation between consecutive (state, action)-pairs. By sampling randomly from this "memory", we break this correlation.

2. Next, two neural networks are initialized with random weights. The network $q(s,a;w)$ is used to approximate the true underlying Q-values. Therefore, if we only use one network, we would end up with the update 

\begin{equation} \Delta w = \alpha(r + \gamma \max_a q(s',a;w) - q(s,a;w))\nabla_w q(s,a;w)  , \end{equation}

which turns out to be unstable, since the weights we are updating are also present in the TD-target (i.e., $r + \gamma \max_a q(s',a;w)$). Therefore, we introduce "fixed Q-targets". This means that during learning, we fix the target, $w'$, and get

\begin{equation} \Delta w = \alpha(r + \gamma \max_a q(s',a;w') - q(s,a;w))\nabla_w q(s,a;w). \end{equation}

3. After initializations, we proceed iteratively:
    * From the given state $s_t$, choose action using an $\epsilon$-greedy strategy and observe reward $r_t$, the next state $s_{t+1}$ and boolean $d_t$ to see if the episode has ended.
    * Store the experience ($s_t$, $a_t$, $r_t$, $s_{t+1}$, $d_t$) in the Replay Buffer.
    * If ``t % UPDATE_EVERY = 0``, we draw a sample of BATCH_SIZE from the Replay Buffer, and take an optimization step according to the equation above.
    * Update the fixed Q-targets after learning, by the soft update 
        \begin{equation} w' = \tau w + (1-\tau) w'. \end{equation}
    This process continues until convergence. In this particular case, the problem is considered solved when the agent obtains an average score above 13 over 100 consecutive episodes.


<style>
.pseudo-code ul {
  list-style-type: none;
}
</style>

* The DDPG algorithm can be summarized as follows:

<nav class="pseudo-code">

```

Randomly initialize critic network $Q(s,a|\theta^Q)$ and actor $\mu(s|\theta^\mu)$ with weights $\theta^Q$ and $\theta^\mu.$

Initialize target network $Q'$ and $\mu'$ with weights $\theta^{Q'} \leftarrow \theta^Q, \theta^{\mu'} \leftarrow \theta^\mu$

Initialize replay buffer $R$

**for** episode = 1, M **do**

  Initialize a random process $\mathcal{N}$ for action exploration
  
  Receive initial observation state $s_1$
  
  **for** t = 1, T **do**
  
   Select action $a_t = \mu(s_t|\theta^\mu) + \mathcal{N}_t$ according to the current policy and exploration noise
   
   Execute action $a_t$ and observe reward $r_t$ and observe new state $s_{t+1}$
   
   Store transition $(s_t, a_t, r_t, s_{t+1})$ in $R$
   
   Sample a random minibatch of $\mathcal{N}$ transitions $(s_i, a_i, r_i, s_{i+1})$ from $R$
   
   Set $y_i = r_i + \gamma Q'(s_{i+1}, \mu'(s_{i+1}|\theta^{\mu'})|\theta^{Q'})$
   
   Update cretic by minimizing the loss: $L = \frac{1}{N} \sum_i(y_i - Q(s_i, a_i|\theta^Q))^2$
   
   Update the actor policy using the sampled policy gradient:
   
   \begin{equation} \nabla_{\theta^\mu} J \approx \frac{1}{N} \sum_i \nabla_a Q(s,a|\theta^Q)|_{s=s_i, a=\mu(s_i)} \nabla_{\theta^\mu}\mu(s|\theta^\mu)|_{s_i}  \end{equation}
   
   Update the target networks:
   
   \begin{align}
   \theta^{Q'} &\leftarrow \tau\theta^Q + (t-\tau) \theta^{Q'}\\
   \theta^{\mu'} &\leftarrow \tau\theta^\mu + (t-\tau)\theta^{\mu'}
   \end{align}

  **end for**

**end for**

```

</nav>

  




### Chosen Hyperparameters

* ``BUFFER_SIZE = int(1e5)``:  Replay buffer size
* ``BATCH_SIZE = 64``:         Minibatch size 
* ``GAMMA = 0.99``:            Discount factor
* ``TAU = 1e-3``:              For soft update of target parameters
* ``LR = 5e-4``:               Learning rate
* ``UPDATE_EVERY = 4``:        How often to update the network


### Neural Network

The neural network used is a simple feed-forward network with fully connected layers:

* Layer 1: (state_size, 64)
* ReLU 1
* Layer 2: (64, 64)
* ReLU 2
* Layer 3: (64, action_size)

Here, state_size = 37 and action_size = 4.

---
header-includes:
  - \usepackage[ruled,vlined,linesnumbered]{algorithm2e}
---

# Algorithm 1
Just a sample algorithmn
\begin{algorithm}[H]
\DontPrintSemicolon
\SetAlgoLined
\KwResult{Write here the result}
\SetKwInOut{Input}{Input}\SetKwInOut{Output}{Output}
\Input{Write here the input}
\Output{Write here the output}
\BlankLine
\While{While condition}{
    instructions\;
    \eIf{condition}{
        instructions1\;
        instructions2\;
    }{
        instructions3\;
    }
}
\caption{While loop with If/Else condition}
\end{algorithm} 

## 2: Plot of Rewards

This setup was able to solve the problem in 462 episodes. A plot of score per episode is illustraded below:

<img src="score-plot.png" width="600">

## 3: Ideas for Future Work

While the problem is solved reasonably quickly with basic Deep Q-learning, the agent could possibly improved in ways including:   

* Search for optimal hyperparameters and neural network size/shape.
* Try extensions of the Q-learning algorithm, including "Double DQN" and "Dueling DQN", and apply prioritized experience replay rather than the current uniform implementation.
* Observe the raw pixels instead (or in addition to) the current ray-based "sensor" inputs. 
